<a href="https://colab.research.google.com/github/aghapygad336/Fake-and-real-news/blob/master/Fake_and_real_news_dataset_Classifying_the_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from collections import Counter

In [32]:
!pip install pyspark==2.4.5
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark = SparkSession \
    .builder \
    .getOrCreate()

     |████████████████████████████████| 217.8MB 33kB/s 
     |████████████████████████████████| 204kB 39.7MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=d5600b4d2053bdfd3a5f5db1faf0486fe95c3cc4260548f499b8e70d360ab0a8
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'bo

In [26]:
def cleanDF(df):
    df.drop(['date', 'title'], axis=1, inplace=True)
    df.text = df.text.str.lower()
    df.text = df.text.str.replace(r'http[\w:/\.]+','<URL>') # remove urls
    df.text = df.text.str.replace(r'[^\.\w\s]','') #remove everything but characters and punctuation
    df.text = df.text.str.replace(r'\.\.+','.') #replace multple periods with a single one
    df.text = df.text.str.replace(r'\.',' . ') #replace multple periods with a single one
    df.text = df.text.str.replace(r'\s\s+',' ') #replace multple white space with a single one
    df.text = df.text.str.strip() 
    print(df.shape)
    df.head()
    return df


In [27]:
True_df = pd.read_csv('/content/drive/My Drive/ask/FakeNews/True.csv')
True_df.head()


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [28]:
True_df=cleanDF(True_df)
True_df.head()


(21417, 2)


,text,subject
0,washington reuters the head of a conservative ...,politicsNews
1,washington reuters transgender people will be ...,politicsNews
2,washington reuters the special counsel investi...,politicsNews
3,washington reuters trump campaign adviser geor...,politicsNews
4,seattlewashington reuters president donald tru...,politicsNews


In [29]:
Fake_df = pd.read_csv('/content/drive/My Drive/ask/FakeNews/Fake.csv')
Fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [30]:
Fake_df=cleanDF(Fake_df)
Fake_df.head()


(23481, 2)


,text,subject
0,donald trump just couldn t wish all americans ...,News
1,house intelligence committee chairman devin nu...,News
2,on friday it was revealed that former milwauke...,News
3,on christmas day donald trump announced that h...,News
4,pope francis used his annual christmas day mes...,News


In [34]:
Fake_df['CLASS'] = 0
Fake_df

,text,subject,CLASS
0,donald trump just couldn t wish all americans ...,News,0
1,house intelligence committee chairman devin nu...,News,0
2,on friday it was revealed that former milwauke...,News,0
3,on christmas day donald trump announced that h...,News,0
4,pope francis used his annual christmas day mes...,News,0
...,...,...,...
23476,21st century wire says as 21wire reported earl...,Middle-east,0
23477,21st century wire says it s a familiar theme ....,Middle-east,0
23478,patrick henningsen 21st century wireremember w...,Middle-east,0
23479,21st century wire says al jazeera america will...,Middle-east,0


In [35]:
True_df['CLASS'] = 1
True_df

,text,subject,CLASS
0,washington reuters the head of a conservative ...,politicsNews,1
1,washington reuters transgender people will be ...,politicsNews,1
2,washington reuters the special counsel investi...,politicsNews,1
3,washington reuters trump campaign adviser geor...,politicsNews,1
4,seattlewashington reuters president donald tru...,politicsNews,1
...,...,...,...
21412,brussels reuters nato allies on tuesday welcom...,worldnews,1
21413,london reuters lexisnexis a provider of legal ...,worldnews,1
21414,minsk reuters in the shadow of disused soviete...,worldnews,1
21415,moscow reuters vatican secretary of state card...,worldnews,1


In [39]:
Fake_df['subject'].unique()

array(['News', 'politics', 'Government News', 'left-news', 'US_News',
       'Middle-east'], dtype=object)